In [ ]:
# ASSNP-13.py
# 2021-11-29
#
from datetime import datetime
dt_start = datetime.now()
#
import os
# to fix matplotlib warning
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
path = os.getcwd()
print('當前工作目錄 ==>', path)
#
import numpy as np
import matplotlib.pyplot as plt
#
path_data = './data/'
myfont = fontManager.FontProperties(fname='/home/jovyan/fonts/msjh.ttc')
#
# -------------------------------------------------------------------------------
#
fig = plt.figure(figsize=(45,70))
plt.xlabel('X座標', fontproperties=myfont, fontsize=26)
plt.ylabel('Y座標', fontproperties=myfont, fontsize=26)
#
plt.xlim(-5, 40)    # xlim, ylim, need to be same
plt.ylim(20, 90)    
#
x_ticks = np.linspace( 0,40,5)
y_ticks = np.linspace(20,90,8)
#
plt.xticks(x_ticks, fontsize=24)
plt.yticks(y_ticks, fontsize=24)
#
plt.grid(False)
plt.gca().set_aspect('equal', adjustable='box') # set X,Y same ratio & scale
#
# 逐筆讀入資料 dem-all-7.csv
# Read from the file into the array data(:,:)
# array data shape is (...., 3)
#
# Columns:
# data[..., 0] is array of float number
# data[..., 1] is array of float number
# data[..., 2] is array of float number
#
# 台灣橫麥卡托二度分帶投影座標系統（TM2）
#
float_To_int = np.vectorize(np.int32)
#
data2 = np.loadtxt(path_data + 'dem-all-7.csv',delimiter=',',dtype='float')
min_item = np.amin(data2, axis=0) # axis=0 --> min of each column
max_item = np.amax(data2, axis=0) # axis=0 --> max of each column
print ('Left Down point:', float_To_int(min_item[0]), float_To_int(min_item[1]))
print ('Right Up point: ' , float_To_int(max_item[0]), float_To_int(max_item[1]))
print ('height (min, max)', min_item[2], max_item[2])
print ('\n')
#
# Left Down point:    0, 2040
# Right  Up point: 3800, 9000
# height (min, max) 0.0, 357.21
#
TM2_X = float_To_int(data2[..., 0]) # 二度分帶 X座標
TM2_Y = float_To_int(data2[..., 1]) # 二度分帶 Y座標
TM2_Z = data2[..., 2]               # DEM file z value, float type
#
# XYlist is sorted (x, y), first order is y, then x
#
XYlist = list(zip(TM2_X,TM2_Y))
#
# X, Y value
X = np.arange(-100, 3820, 20)
Y = np.arange(2040, 9020, 20)
Xmesh, Ymesh = np.meshgrid(X, Y)  # x-y 平面的網格
#
# X -100 .. 3800
# Y 2040 .. 9000
# 6980/20 ==> 349
# 3920/20 ==> 196
#
Zmesh = np.zeros(shape=(349,196),dtype=np.float16)
i = 0 ; j = 0
#
for y in np.arange(2040, 9020, 20) :
    print('y is', y)
    z = np.zeros(shape=(196),dtype=np.float16)
    zi = 0
    for x in np.arange(-100, 3820, 20):
        if (x,y) in XYlist :
            z[zi] = TM2_Z[i]
            i = i + 1
        else :
            z[zi] = 0.0
        # end if
        zi = zi + 1
    # end for
    Zmesh[j] = z
    j = j + 1
#end for
print ('\n')
#
factor_3D = 0.01
Xmesh_3D = factor_3D * Xmesh
Ymesh_3D = factor_3D * Ymesh
Zmesh_3C = np.around(Zmesh)
Zmesh_3D = Zmesh_3C.astype(int)
#
# If array-like, draw contour lines at the specified levels. The values must be in increasing order.
#
contour_set = plt.contour(Xmesh_3D, Ymesh_3D, Zmesh_3D, [15])
plt.draw()
# return paired points making up each line segment
path_set = contour_set.collections[0].get_paths()
# To get a NumPy array of the coordinates, use the Path.vertices attribute.
print("path_set total segments is", len(path_set))
j = 1
for path in path_set :
    # grab the path coordinates
    path_coords = path.vertices
    print('path', j)
    xy_list = []
    for xy in path_coords :
        xy_list.append(tuple([round(xy[0],3),round(xy[1],3)]))
    # end for
    print(xy_list)
    print('====================================')
    #
    j = j + 1
# end for
#
print(dt_start)
dt_end = datetime.now()
print(dt_end)
#
print('done')
#